I'll continue with **Chapter 23: Platform Channels** following the comprehensive structure and industry standards established in the previous chapter.

---

# **Chapter 23: Platform Channels**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Establish bidirectional communication between Dart and native platform code (Android/iOS)
- Implement MethodChannel for invoking platform methods from Flutter
- Use EventChannel for streaming data from platform to Flutter
- Configure BasicMessageChannel with custom codecs for structured data
- Embed native platform views within Flutter widgets using PlatformView
- Integrate external textures for camera, video, and OpenGL content
- Handle platform channel errors and exceptions gracefully
- Architect platform channel code following industry best practices

---

## **Prerequisites**

- Completed Chapter 22: Local Data Persistence
- Understanding of async/await and Streams in Dart
- Basic familiarity with Android (Kotlin/Java) or iOS (Swift/Objective-C) development
- Flutter development environment configured for Android and iOS

---

## **23.1 Introduction to Platform Channels**

Flutter provides a platform channel mechanism for communicating between Dart code and platform-specific native code (Android Kotlin/Java, iOS Swift/Objective-C, Windows C++, macOS Swift/Objective-C, Linux C++). This is essential when you need to access platform-specific APIs that aren't available in Flutter's plugin ecosystem.

### **Architecture Overview**

```dart
// Conceptual diagram of platform channel architecture
/*
┌─────────────────────────────────────────────────────────────┐
│                    Flutter (Dart)                             │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │  MethodChannel│  │  EventChannel│  │BasicMessage  │      │
│  │   (invoke)   │  │   (stream)   │  │   Channel    │      │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘      │
└─────────┼─────────────────┼─────────────────┼──────────────┘
          │                 │                 │
          └─────────────────┼─────────────────┘
                            │
┌───────────────────────────┼─────────────────────────────────┐
│              Platform (Android/iOS)                           │
│  ┌────────────────────────┴──────────────────────────────┐ │
│  │              Platform Channel Handler                    │ │
│  │  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  │ │
│  │  │  MethodCall  │  │  EventSink   │  │  MessageCodec│  │ │
│  │  │   Handler    │  │   Handler    │  │   Handler    │  │ │
│  │  └──────────────┘  └──────────────┘  └──────────────┘  │ │
│  └─────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────┘
*/
```

**Explanation:**

- **Platform Channels**: The bridge between Dart and native code. Three main types:
  - **MethodChannel**: For invoking methods bidirectionally (request-response pattern)
  - **EventChannel**: For streaming data from platform to Dart (event streams)
  - **BasicMessageChannel**: For passing data using custom codecs (message passing)
- **Platform Thread**: Native code runs on the platform's main thread (UI thread) by default. Heavy operations should be moved to background threads.
- **Binary Messenger**: The low-level communication layer that serializes/deserializes messages between Dart and platform.

---

## **23.2 MethodChannel**

MethodChannel is the most commonly used platform channel. It allows you to invoke platform methods from Dart and receive results asynchronously.

### **Dart Implementation**

```dart
import 'package:flutter/services.dart';

class BatteryService {
  // Create a MethodChannel with a unique name
  // The name should be in reverse domain notation to avoid conflicts
  static const platform = MethodChannel('com.example.app/battery');
  // MethodChannel constructor takes a String channel name
  // This name must match exactly on both Dart and platform sides
  
  /// Gets the battery level from the platform
  /// Returns the battery level as an integer percentage (0-100)
  /// Throws a PlatformException if the platform call fails
  static Future<int> getBatteryLevel() async {
    try {
      // Invoke a method on the platform
      // 'getBatteryLevel' is the method name to call on the platform side
      final int result = await platform.invokeMethod('getBatteryLevel');
      // invokeMethod returns a Future<dynamic>
      // We cast it to int (platform must return an int)
      
      return result;
    } on PlatformException catch (e) {
      // PlatformException is thrown when the platform side encounters an error
      // e.code: Error code string from platform
      // e.message: Human-readable error message
      // e.details: Additional error data
      
      print('Failed to get battery level: ${e.message}');
      throw BatteryException('Failed to get battery level: ${e.message}');
    } on MissingPluginException catch (e) {
      // Thrown when the platform side hasn't implemented the method
      print('Plugin not implemented: ${e.message}');
      throw BatteryException('Battery plugin not implemented on this platform');
    }
  }
  
  /// Sends data to the platform and receives a response
  static Future<String> sendData(Map<String, dynamic> data) async {
    try {
      // Pass arguments to the platform method
      // Arguments must be JSON-serializable types:
      // null, bool, int, double, String, List, or Map
      
      final String result = await platform.invokeMethod(
        'processData',
        data,  // Arguments passed as the second parameter
      );
      
      return result;
    } on PlatformException catch (e) {
      return 'Error: ${e.message}';
    }
  }
}

// Custom exception class
class BatteryException implements Exception {
  final String message;
  BatteryException(this.message);
  
  @override
  String toString() => 'BatteryException: $message';
}
```

**Explanation:**

- **MethodChannel**: Created with a unique channel name string. This name must match exactly on both Dart and platform sides.
- **`invokeMethod`**: Asynchronously calls a method on the platform. Takes a method name string and optional arguments.
- **Arguments**: Must be JSON-serializable (null, bool, int, double, String, List, Map).
- **Return values**: Platform returns JSON-serializable values that are automatically converted to Dart types.
- **Error handling**: `PlatformException` for platform errors, `MissingPluginException` for unimplemented methods.
- **Type safety**: Cast the result of `invokeMethod` to the expected type.

### **Android Implementation (Kotlin)**

```kotlin
package com.example.app

import android.content.Context
import android.content.Context.BATTERY_SERVICE
import android.os.BatteryManager
import android.os.Build
import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.MethodChannel

class MainActivity : FlutterActivity() {
    // Channel name must match exactly with Dart side
    private val CHANNEL = "com.example.app/battery"
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        // Create MethodChannel and set handler
        MethodChannel(flutterEngine.dartExecutor.binaryMessenger, CHANNEL)
            .setMethodCallHandler { call, result ->
                // call.method contains the method name from Dart
                // call.arguments contains the arguments passed from Dart
                // result is used to return data or errors to Dart
                
                when (call.method) {
                    "getBatteryLevel" -> {
                        val batteryLevel = getBatteryLevel()
                        
                        if (batteryLevel != -1) {
                            // Success: return the battery level
                            result.success(batteryLevel)
                        } else {
                            // Error: return error details
                            result.error(
                                "UNAVAILABLE",  // Error code
                                "Battery level not available.",  // Error message
                                null  // Error details (optional)
                            )
                        }
                    }
                    "processData" -> {
                        // Extract arguments
                        val data = call.arguments as? Map<String, Any>
                        
                        if (data != null) {
                            val processed = processData(data)
                            result.success(processed)
                        } else {
                            result.error(
                                "INVALID_ARGUMENT",
                                "Expected Map<String, dynamic>",
                                null
                            )
                        }
                    }
                    else -> {
                        // Method not implemented
                        result.notImplemented()
                    }
                }
            }
    }
    
    private fun getBatteryLevel(): Int {
        return if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.LOLLIPOP) {
            val batteryManager = getSystemService(Context.BATTERY_SERVICE) as BatteryManager
            batteryManager.getIntProperty(BatteryManager.BATTERY_PROPERTY_CAPACITY)
        } else {
            -1
        }
    }
    
    private fun processData(data: Map<String, Any>): String {
        // Process the data and return result
        return "Processed: ${data.size} items"
    }
}
```

**Explanation:**

- **MethodChannel setup**: Created with the same channel name as Dart side, using `flutterEngine.dartExecutor.binaryMessenger`.
- **`setMethodCallHandler`**: Sets a callback that receives method calls from Dart.
- **`call.method`**: The string method name sent from Dart via `invokeMethod`.
- **`call.arguments`**: The arguments passed from Dart (automatically converted to Kotlin types).
- **`result.success()`**: Returns a value to Dart (completes the Future).
- **`result.error()`**: Returns an error to Dart (throws PlatformException in Dart).
- **`result.notImplemented()`**: Indicates the method is not implemented (throws MissingPluginException in Dart).
- **Type casting**: Use `as?` for safe casting of arguments to expected types.

### **iOS Implementation (Swift)**

```swift
import UIKit
import Flutter

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
    // Channel name must match Dart side exactly
    private let CHANNEL = "com.example.app/battery"
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        // Get the root view controller (FlutterViewController)
        let controller = window?.rootViewController as! FlutterViewController
        
        // Create MethodChannel
        let batteryChannel = FlutterMethodChannel(
            name: CHANNEL,
            binaryMessenger: controller.binaryMessenger
        )
        
        // Set method call handler
        batteryChannel.setMethodCallHandler { [weak self] (call, result) in
            // call.method: String method name from Dart
            // call.arguments: Arguments from Dart (Any?)
            // result: FlutterResult callback to return data
            
            guard let self = self else { return }
            
            switch call.method {
            case "getBatteryLevel":
                let batteryLevel = self.getBatteryLevel()
                
                if batteryLevel != -1 {
                    // Success: return battery level
                    result(batteryLevel)
                } else {
                    // Error: return error details
                    result(FlutterError(
                        code: "UNAVAILABLE",
                        message: "Battery level not available",
                        details: nil
                    ))
                }
                
            case "processData":
                // Extract arguments
                if let data = call.arguments as? [String: Any] {
                    let processed = self.processData(data: data)
                    result(processed)
                } else {
                    result(FlutterError(
                        code: "INVALID_ARGUMENT",
                        message: "Expected Map<String, dynamic>",
                        details: nil
                    ))
                }
                
            default:
                // Method not implemented
                result(FlutterMethodNotImplemented)
            }
        }
        
        GeneratedPluginRegistrant.register(with: self)
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    private func getBatteryLevel() -> Int {
        let device = UIDevice.current
        device.isBatteryMonitoringEnabled = true
        
        if device.batteryState == .unknown {
            return -1
        }
        
        return Int(device.batteryLevel * 100)
    }
    
    private func processData(data: [String: Any]) -> String {
        return "Processed: \(data.count) items"
    }
}
```

**Explanation:**

- **FlutterMethodChannel**: Created with the same channel name as Dart, using the `binaryMessenger` from the `FlutterViewController`.
- **`setMethodCallHandler`**: Sets a closure that handles method calls from Dart.
- **`call.method`**: The method name string sent from Dart.
- **`call.arguments`**: Arguments passed from Dart (cast to appropriate Swift types).
- **`result(value)`**: Returns a successful result to Dart (completes the Future).
- **`FlutterError`**: Returns an error to Dart (throws PlatformException).
- **`FlutterMethodNotImplemented`**: Indicates the method is not implemented (throws MissingPluginException).
- **Memory management**: Use `[weak self]` in closures to avoid retain cycles.

---

## **23.3 EventChannel**

EventChannel is used for streaming data from the platform to Dart. It's ideal for continuous data streams like sensor readings, location updates, or native event streams.

### **Dart Implementation**

```dart
import 'package:flutter/services.dart';
import 'dart:async';

class SensorService {
  // EventChannel for streaming accelerometer data
  static const EventChannel _accelerometerChannel = 
      EventChannel('com.example.app/sensors/accelerometer');
  // EventChannel takes a unique name string, similar to MethodChannel
  
  // EventChannel for gyroscope data
  static const EventChannel _gyroscopeChannel = 
      EventChannel('com.example.app/sensors/gyroscope');
  
  // Stream subscriptions to manage lifecycle
  StreamSubscription? _accelerometerSubscription;
  StreamSubscription? _gyroscopeSubscription;
  
  /// Starts listening to accelerometer events
  /// Returns a Stream of Map<String, double> containing x, y, z values
  Stream<Map<String, double>> get accelerometerStream {
    // receiveBroadcastStream returns a Stream<dynamic>
    // We map it to our desired type
    return _accelerometerChannel
        .receiveBroadcastStream()
        // receiveBroadcastStream starts listening to the platform stream
        // It accepts optional arguments: receiveBroadcastStream([arguments])
        .map((event) {
          // Cast the dynamic event to Map<dynamic, dynamic>
          final Map<dynamic, dynamic> map = event;
          
          // Convert to typed Map<String, double>
          return {
            'x': (map['x'] as num).toDouble(),
            'y': (map['y'] as num).toDouble(),
            'z': (map['z'] as num).toDouble(),
          };
        });
  }
  
  /// Starts listening to sensor data with error handling
  void startListening({
    required Function(Map<String, double>) onAccelerometer,
    required Function(String) onError,
  }) {
    // Cancel any existing subscriptions
    _accelerometerSubscription?.cancel();
    
    // Listen to the stream
    _accelerometerSubscription = accelerometerStream.listen(
      (data) {
        // Called for each event from platform
        onAccelerometer(data);
      },
      onError: (error) {
        // Called when platform sends an error
        onError('Sensor error: $error');
      },
      onDone: () {
        // Called when platform closes the stream
        print('Sensor stream closed');
      },
      cancelOnError: false,  // Don't cancel subscription on error
    );
  }
  
  /// Stops listening to sensors
  void stopListening() {
    _accelerometerSubscription?.cancel();
    _accelerometerSubscription = null;
  }
}

// Usage in a Widget
class SensorPage extends StatefulWidget {
  @override
  _SensorPageState createState() => _SensorPageState();
}

class _SensorPageState extends State<SensorPage> {
  final SensorService _sensorService = SensorService();
  Map<String, double>? _accelerometerData;
  String? _error;
  
  @override
  void initState() {
    super.initState();
    _sensorService.startListening(
      onAccelerometer: (data) {
        // Update UI with new sensor data
        if (mounted) {
          setState(() {
            _accelerometerData = data;
            _error = null;
          });
        }
      },
      onError: (error) {
        if (mounted) {
          setState(() => _error = error);
        }
      },
    );
  }
  
  @override
  void dispose() {
    _sensorService.stopListening();
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Accelerometer')),
      body: Center(
        child: _error != null
            ? Text('Error: $_error', style: TextStyle(color: Colors.red))
            : _accelerometerData == null
                ? CircularProgressIndicator()
                : Column(
                    mainAxisAlignment: MainAxisAlignment.center,
                    children: [
                      Text('X: ${_accelerometerData!['x']?.toStringAsFixed(2)}'),
                      Text('Y: ${_accelerometerData!['y']?.toStringAsFixed(2)}'),
                      Text('Z: ${_accelerometerData!['z']?.toStringAsFixed(2)}'),
                    ],
                  ),
      ),
    );
  }
}
```

**Explanation:**

- **EventChannel**: Created with a unique name string, similar to MethodChannel but for streaming data.
- **`receiveBroadcastStream()`**: Returns a `Stream<dynamic>` that receives events from the platform. Can optionally take arguments to pass to the platform when starting the stream.
- **Stream mapping**: The raw stream returns `dynamic` data (usually Map or List), which needs to be cast and transformed to typed Dart objects.
- **Subscription management**: Always store `StreamSubscription` references to cancel them when the widget disposes to prevent memory leaks.
- **Error handling**: The stream can emit errors from the platform side. Always handle `onError` callbacks.
- **Lifecycle**: Start listening in `initState`, cancel in `dispose`. Check `mounted` before calling `setState` to avoid errors after disposal.

### **Android Implementation (Kotlin)**

```kotlin
package com.example.app

import android.content.Context
import android.hardware.Sensor
import android.hardware.SensorEvent
import android.hardware.SensorEventListener
import android.hardware.SensorManager
import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.EventChannel

class MainActivity : FlutterActivity() {
    // Channel name must match Dart side exactly
    private val ACCELEROMETER_CHANNEL = "com.example.app/sensors/accelerometer"
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        // Setup EventChannel with stream handler
        EventChannel(flutterEngine.dartExecutor.binaryMessenger, ACCELEROMETER_CHANNEL)
            .setStreamHandler(AccelerometerStreamHandler(context))
        // setStreamHandler registers a handler that manages the stream lifecycle
    }
}

// StreamHandler implementation for accelerometer
class AccelerometerStreamHandler(private val context: Context) : EventChannel.StreamHandler {
    private var sensorManager: SensorManager? = null
    private var accelerometer: Sensor? = null
    private var eventSink: EventChannel.EventSink? = null
    private var sensorListener: SensorEventListener? = null
    
    // Called when the first listener subscribes to the stream in Dart
    override fun onListen(arguments: Any?, events: EventChannel.EventSink?) {
        eventSink = events
        // events is the sink used to send data to Dart
        
        setupSensor()
    }
    
    // Called when the last listener unsubscribes from the stream
    override fun onCancel(arguments: Any?) {
        // Clean up resources to prevent memory leaks
        sensorListener?.let { sensorManager?.unregisterListener(it) }
        sensorListener = null
        eventSink = null
        sensorManager = null
    }
    
    private fun setupSensor() {
        sensorManager = context.getSystemService(Context.SENSOR_SERVICE) as SensorManager
        accelerometer = sensorManager?.getDefaultSensor(Sensor.TYPE_ACCELEROMETER)
        
        if (accelerometer == null) {
            // Send error to Dart if sensor not available
            eventSink?.error(
                "SENSOR_NOT_AVAILABLE",
                "Accelerometer not available on this device",
                null
            )
            return
        }
        
        // Create sensor event listener
        sensorListener = object : SensorEventListener {
            override fun onSensorChanged(event: SensorEvent?) {
                event?.let {
                    // Create map to send to Dart
                    // Must match the structure expected in Dart
                    val data = mapOf(
                        "x" to it.values[0],  // Acceleration on X axis
                        "y" to it.values[1],  // Acceleration on Y axis
                        "z" to it.values[2]   // Acceleration on Z axis
                    )
                    
                    // Send data to Dart through the event sink
                    eventSink?.success(data)
                    // success sends a data event to the Dart stream
                }
            }
            
            override fun onAccuracyChanged(sensor: Sensor?, accuracy: Int) {
                // Handle accuracy changes if needed
                // Can send accuracy updates to Dart if required
            }
        }
        
        // Register listener with sensor manager
        // SENSOR_DELAY_NORMAL is the update rate (can be UI, GAME, FASTEST)
        sensorManager?.registerListener(
            sensorListener,
            accelerometer,
            SensorManager.SENSOR_DELAY_NORMAL
        )
    }
}
```

**Explanation:**

- **EventChannel.StreamHandler**: Interface with two methods: `onListen` (when Dart starts listening) and `onCancel` (when Dart stops listening).
- **EventSink**: The object used to send events to Dart. Use `success(data)` for data events, `error(code, message, details)` for errors, and `endOfStream()` to close the stream.
- **Lifecycle management**: Always clean up resources (sensors, listeners, timers) in `onCancel` to prevent memory leaks and battery drain.
- **Data format**: Data sent through EventSink must be JSON-serializable (Map, List, primitive types).
- **Error handling**: Send platform errors through `eventSink.error()` which will be received as errors in the Dart Stream.

### **iOS Implementation (Swift)**

```swift
import UIKit
import Flutter
import CoreMotion  // For accelerometer access

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
    // Channel name must match Dart side exactly
    private let ACCELEROMETER_CHANNEL = "com.example.app/sensors/accelerometer"
    private var motionManager: CMMotionManager?
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        let controller = window?.rootViewController as! FlutterViewController
        
        // Create EventChannel
        let accelerometerChannel = FlutterEventChannel(
            name: ACCELEROMETER_CHANNEL,
            binaryMessenger: controller.binaryMessenger
        )
        
        // Set stream handler
        accelerometerChannel.setStreamHandler(AccelerometerStreamHandler())
        
        GeneratedPluginRegistrant.register(with: self)
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
}

// StreamHandler implementation
class AccelerometerStreamHandler: NSObject, FlutterStreamHandler {
    private var motionManager: CMMotionManager?
    private var eventSink: FlutterEventSink?
    
    // Called when Dart starts listening to the stream
    func onListen(
        withArguments arguments: Any?,
        eventSink events: @escaping FlutterEventSink
    ) -> FlutterError? {
        
        self.eventSink = events
        // events is a closure that sends data to Dart
        
        setupAccelerometer()
        return nil  // Return nil for success, FlutterError for error
    }
    
    // Called when Dart cancels the stream subscription
    func onCancel(withArguments arguments: Any?) -> FlutterError? {
        // Clean up resources
        motionManager?.stopAccelerometerUpdates()
        motionManager = nil
        eventSink = nil
        return nil
    }
    
    private func setupAccelerometer() {
        motionManager = CMMotionManager()
        
        guard let manager = motionManager else {
            eventSink?(
                FlutterError(
                    code: "SENSOR_NOT_AVAILABLE",
                    message: "Motion manager not available",
                    details: nil
                )
            )
            return
        }
        
        // Check if accelerometer is available
        guard manager.isAccelerometerAvailable else {
            eventSink?(
                FlutterError(
                    code: "ACCELEROMETER_NOT_AVAILABLE",
                    message: "Accelerometer not available on this device",
                    details: nil
                )
            )
            return
        }
        
        // Set update interval (in seconds)
        manager.accelerometerUpdateInterval = 0.1  // 100ms
        
        // Start accelerometer updates
        manager.startAccelerometerUpdates(to: .main) { [weak self] (data, error) in
            // data is CMAccelerometerData?
            // error is Error?
            
            guard let self = self, let eventSink = self.eventSink else { return }
            
            if let error = error {
                eventSink(
                    FlutterError(
                        code: "SENSOR_ERROR",
                        message: error.localizedDescription,
                        details: nil
                    )
                )
                return
            }
            
            guard let accelerometerData = data else { return }
            
            // Create dictionary to send to Dart
            // Must match the structure expected in Dart
            let event: [String: Any] = [
                "x": accelerometerData.acceleration.x,
                "y": accelerometerData.acceleration.y,
                "z": accelerometerData.acceleration.z
            ]
            
            // Send to Dart
            eventSink(event)
        }
    }
}
```

**Explanation:**

- **FlutterEventChannel**: Created with the same name as Dart side, using the `binaryMessenger` from `FlutterViewController`.
- **FlutterStreamHandler**: Protocol (interface) with two methods: `onListen` (when Dart subscribes) and `onCancel` (when Dart unsubscribes).
- **FlutterEventSink**: Closure used to send events to Dart. Call it with data for success, or with `FlutterError` for errors.
- **Resource cleanup**: Stop sensor updates and nil out references in `onCancel` to prevent memory leaks and battery drain.
- **Error handling**: Send `FlutterError` objects through the event sink for platform errors.
- **Threading**: By default, callbacks run on the main thread. For heavy processing, dispatch to background queues.

---

## **23.4 BasicMessageChannel and Custom Codecs**

BasicMessageChannel allows you to send and receive messages using custom codecs. This is useful when you need more control over serialization than the standard platform channels provide.

### **Dart Implementation**

```dart
import 'package:flutter/services.dart';
import 'dart:convert';

class MessageService {
  // StandardMessageCodec uses binary serialization
  // Supports null, bool, int, double, String, Uint8List, Int32List, Int64List, Float64List, List, Map
  static const BasicMessageChannel<dynamic> _standardChannel = 
      BasicMessageChannel(
    'com.example.app/message/standard',
    StandardMessageCodec(),
  );
  
  // JSONMessageCodec for JSON serialization
  static const BasicMessageChannel<dynamic> _jsonChannel = 
      BasicMessageChannel(
    'com.example.app/message/json',
    JSONMessageCodec(),
  );
  
  // Binary codec for raw bytes
  static const BasicMessageChannel<Uint8List> _binaryChannel = 
      BasicMessageChannel(
    'com.example.app/message/binary',
    BinaryCodec(),
  );
  
  /// Sends a message using StandardMessageCodec
  /// Automatically handles basic types
  static Future<dynamic> sendStandardMessage(String message) async {
    try {
      final response = await _standardChannel.send(message);
      // send() returns a Future<dynamic> with the platform's response
      
      print('Received: $response');
      return response;
    } catch (e) {
      print('Error: $e');
      rethrow;
    }
  }
  
  /// Sends structured data using StandardMessageCodec
  static Future<dynamic> sendStructuredData() async {
    final data = {
      'type': 'user_action',
      'timestamp': DateTime.now().millisecondsSinceEpoch,
      'data': {
        'userId': 12345,
        'action': 'button_click',
        'metadata': ['tag1', 'tag2'],
      },
    };
    
    final response = await _standardChannel.send(data);
    return response;
  }
  
  /// Sends JSON data using JSONMessageCodec
  static Future<dynamic> sendJsonMessage(Map<String, dynamic> data) async {
    // JSONMessageCodec automatically handles JSON serialization
    final response = await _jsonChannel.send(data);
    
    if (response is Map) {
      return response;
    }
    return null;
  }
  
  /// Sends binary data using BinaryCodec
  static Future<Uint8List?> sendBinaryData(Uint8List data) async {
    // BinaryCodec passes raw bytes without modification
    final response = await _binaryChannel.send(data);
    return response;
  }
  
  /// Sets up a message handler to receive messages from platform
  static void setupMessageHandler() {
    _standardChannel.setMessageHandler((message) async {
      // This handler is called when platform sends a message to Dart
      print('Received from platform: $message');
      
      // Return a response to the platform
      return 'Acknowledged: $message';
    });
  }
}
```

**Explanation:**

- **BasicMessageChannel**: Used for sending messages with specific codecs. Unlike MethodChannel, it's bidirectional by default (both sides can send messages).
- **StandardMessageCodec**: Supports null, bool, int, double, String, Uint8List, Int32List, Int64List, Float64List, List, and Map. Automatically handles type conversion.
- **JSONMessageCodec**: Uses JSON encoding for messages. Good for interoperability with web services.
- **BinaryCodec**: Passes raw bytes without modification. Use for images, audio, or custom binary protocols.
- **`send()`**: Sends a message and returns a Future with the response.
- **`setMessageHandler()`**: Sets a callback to receive messages from the platform.

### **Android Implementation (Kotlin)**

```kotlin
package com.example.app

import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.BasicMessageChannel
import io.flutter.plugin.common.StandardMessageCodec

class MainActivity : FlutterActivity() {
    private val STANDARD_CHANNEL = "com.example.app/message/standard"
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        // Create BasicMessageChannel with StandardMessageCodec
        val messageChannel = BasicMessageChannel(
            flutterEngine.dartExecutor.binaryMessenger,
            STANDARD_CHANNEL,
            StandardMessageCodec.INSTANCE
        )
        // StandardMessageCodec.INSTANCE is the singleton instance
        
        // Set message handler to receive messages from Dart
        messageChannel.setMessageHandler { message, reply ->
            // message: The data sent from Dart (Any?)
            // reply: BasicMessageChannel.Reply callback to send response
            
            println("Received from Dart: $message")
            
            // Process the message
            val response = when (message) {
                is String -> "Received string: $message"
                is Map<*, *> -> "Received map with ${message.size} entries"
                else -> "Received: $message"
            }
            
            // Send reply back to Dart
            reply.reply(response)
            // reply.reply() sends the response back to Dart's Future
        }
    }
}
```

**Explanation:**

- **BasicMessageChannel**: Created with binary messenger, channel name, and codec.
- **StandardMessageCodec.INSTANCE**: The singleton instance of the standard codec.
- **`setMessageHandler`**: Sets a callback that receives messages from Dart. The callback receives the message and a `reply` object.
- **`reply.reply()`**: Sends a response back to Dart, completing the Future from `send()`.

### **iOS Implementation (Swift)**

```swift
import UIKit
import Flutter

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
    private let STANDARD_CHANNEL = "com.example.app/message/standard"
    
    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        
        let controller = window?.rootViewController as! FlutterViewController
        
        // Create BasicMessageChannel
        let messageChannel = FlutterBasicMessageChannel(
            name: STANDARD_CHANNEL,
            binaryMessenger: controller.binaryMessenger,
            codec: FlutterStandardMessageCodec.sharedInstance()
        )
        
        // Set message handler
        messageChannel.setMessageHandler { [weak self] (message, reply) in
            guard let self = self else { return }
            
            print("Received from Dart: \(String(describing: message))")
            
            // Process message and send reply
            let response = self.processMessage(message)
            
            // Send response back to Dart
            reply?(response)
            // reply is optional closure that sends response
        }
        
        GeneratedPluginRegistrant.register(with: self)
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    private func processMessage(_ message: Any?) -> Any {
        switch message {
        case let str as String:
            return "Received string: \(str)"
        case let dict as [String: Any]:
            return "Received dict with \(dict.count) entries"
        default:
            return "Received: \(String(describing: message))"
        }
    }
}
```

**Explanation:**

- **FlutterBasicMessageChannel**: Created with name, binary messenger, and codec.
- **`setMessageHandler`**: Sets a closure to receive messages. The closure receives the message and a `reply` callback.
- **`reply?(response)`**: Sends the response back to Dart. The reply is optional (can be nil).
- **Message processing**: Handle different message types (String, Dictionary, etc.) and return appropriate responses.

---

## **23.4 PlatformView**

PlatformView allows you to embed native platform views (Android Views or iOS UIViews) inside your Flutter widget tree. This is essential when you need to use native components that don't have Flutter equivalents (e.g., Google Maps, WebView before webview_flutter, camera previews).

### **Android Implementation (Kotlin)**

```kotlin
package com.example.app

import android.content.Context
import android.view.View
import android.widget.TextView
import io.flutter.plugin.platform.PlatformView

// Native Android View to be embedded in Flutter
class NativeTextView(context: Context, id: Int, creationParams: Map<String, Any>?) : PlatformView {
    // PlatformView interface requires implementing getView()
    
    private val textView: TextView = TextView(context)
    // The actual Android View instance
    
    init {
        // Apply creation parameters from Flutter
        textView.textSize = 20f
        
        creationParams?.let { params ->
            val text = params["text"] as? String ?: "Default Text"
            val textColor = params["textColor"] as? String ?: "#000000"
            
            textView.text = text
            textView.setTextColor(android.graphics.Color.parseColor(textColor))
        }
        
        // Set layout parameters
        textView.layoutParams = View.LayoutParams(
            View.LayoutParams.MATCH_PARENT,
            View.LayoutParams.WRAP_CONTENT
        )
    }
    
    // Return the Android View to be embedded in Flutter
    override fun getView(): View {
        return textView
    }
    
    // Called when the PlatformView is disposed
    override fun dispose() {
        // Clean up resources
        textView.text = null
    }
}

// Factory to create instances of NativeTextView
class NativeTextViewFactory : PlatformViewFactory(StandardMessageCodec.INSTANCE) {
    // PlatformViewFactory requires implementing create()
    
    override fun create(context: Context, viewId: Int, args: Any?): PlatformView {
        // context: Android Context
        // viewId: Unique ID for this view instance
        // args: Creation arguments from Dart (decoded by the codec)
        
        val creationParams = args as? Map<String, Any>?
        return NativeTextView(context, viewId, creationParams)
    }
}
```

**Explanation:**

- **PlatformView**: Interface that wraps a native Android View. Must implement `getView()` (returns the View) and `dispose()` (cleanup).
- **Creation parameters**: Data passed from Flutter during view creation (e.g., initial text, colors). Received as a Map in `create()`.
- **PlatformViewFactory**: Factory class that creates PlatformView instances. Registered with Flutter to handle view creation requests.
- **StandardMessageCodec**: Default codec for passing simple data structures (Map, List, primitives) between Dart and platform.
- **View lifecycle**: `create()` → `getView()` → ... → `dispose()`. The view is embedded in Flutter's view hierarchy but managed by the platform.

### **iOS Implementation (Swift)**

```swift
import UIKit
import Flutter

// Native iOS View to be embedded in Flutter
class NativeLabelView: NSObject, FlutterPlatformView {
    // FlutterPlatformView protocol requires view() method
    
    private var label: UILabel
    private var channel: FlutterMethodChannel?
    // Method channel for communicating with this specific view instance
    
    init(frame: CGRect, viewId: Int64, args: Any?, messenger: FlutterBinaryMessenger) {
        // frame: Initial frame (usually CGRect.zero, Flutter handles sizing)
        // viewId: Unique identifier for this view instance
        // args: Creation arguments from Dart
        // messenger: Binary messenger for creating method channels
        
        label = UILabel()
        super.init()
        
        // Setup label appearance
        label.textAlignment = .center
        label.textColor = .black
        label.font = UIFont.systemFont(ofSize: 20)
        
        // Apply creation parameters
        if let params = args as? [String: Any] {
            if let text = params["text"] as? String {
                label.text = text
            }
            if let colorHex = params["textColor"] as? String {
                label.textColor = UIColor(hexString: colorHex)
            }
        }
        
        // Setup method channel for this specific view
        // Channel name includes viewId to make it unique per instance
        let channelName = "com.example.app/native_view/\(viewId)"
        channel = FlutterMethodChannel(
            name: channelName,
            binaryMessenger: messenger
        )
        
        // Handle method calls from Dart
        channel?.setMethodCallHandler { [weak self] (call, result) in
            guard let self = self else { return }
            
            switch call.method {
            case "setText":
                if let args = call.arguments as? [String: Any],
                   let text = args["text"] as? String {
                    self.label.text = text
                    result(nil)  // Success, return nil
                } else {
                    result(FlutterError(
                        code: "INVALID_ARGUMENT",
                        message: "Expected Map with 'text' key",
                        details: nil
                    ))
                }
                
            case "getText":
                result(self.label.text)
                
            case "setTextColor":
                if let args = call.arguments as? [String: Any],
                   let colorHex = args["color"] as? String {
                    self.label.textColor = UIColor(hexString: colorHex)
                    result(nil)
                } else {
                    result(FlutterError(code: "INVALID_ARGUMENT", message: "Invalid color", details: nil))
                }
                
            default:
                result(FlutterMethodNotImplemented)
            }
        }
    }
    
    // Return the UIView to be embedded in Flutter
    func view() -> UIView {
        return label
    }
    
    // Cleanup when view is disposed
    deinit {
        channel?.setMethodCallHandler(nil)
    }
}

// Factory to create NativeLabelView instances
class NativeLabelViewFactory: NSObject, FlutterPlatformViewFactory {
    private var messenger: FlutterBinaryMessenger
    
    init(messenger: FlutterBinaryMessenger) {
        self.messenger = messenger
        super.init()
    }
    
    // Create view instance
    func create(
        withFrame frame: CGRect,
        viewIdentifier viewId: Int64,
        arguments args: Any?
    ) -> FlutterPlatformView {
        return NativeLabelView(
            frame: frame,
            viewId: viewId,
            args: args,
            messenger: messenger
        )
    }
}

// Helper extension for UIColor from hex string
extension UIColor {
    convenience init(hexString: String) {
        let hex = hexString.trimmingCharacters(in: CharacterSet.alphanumerics.inverted)
        var int = UInt64()
        Scanner(string: hex).scanHexInt64(&int)
        let a, r, g, b: UInt64
        switch hex.count {
        case 3: // RGB (12-bit)
            (a, r, g, b) = (255, (int >> 8) * 17, (int >> 4 & 0xF) * 17, (int & 0xF) * 17)
        case 6: // RGB (24-bit)
            (a, r, g, b) = (255, int >> 16, int >> 8 & 0xFF, int & 0xFF)
        case 8: // ARGB (32-bit)
            (a, r, g, b) = (int >> 24, int >> 16 & 0xFF, int >> 8 & 0xFF, int & 0xFF)
        default:
            (a, r, g, b) = (255, 0, 0, 0)
        }
        self.init(red: CGFloat(r) / 255, green: CGFloat(g) / 255, blue: CGFloat(b) / 255, alpha: CGFloat(a) / 255)
    }
}
```

**Explanation:**

- **FlutterPlatformView**: Protocol that requires a `view()` method returning the native UIView.
- **View-specific MethodChannel**: Create a MethodChannel unique to each view instance (include viewId in channel name) to control individual native views from Dart.
- **Factory pattern**: `FlutterPlatformViewFactory` creates instances of your platform view when Flutter requests them via `PlatformViewLink` or `AndroidView`/`UiKitView`.
- **Method handlers**: Set up method call handlers to receive commands from Dart (e.g., `setText`, `setTextColor`).
- **Resource cleanup**: Remove method handlers in `deinit` to prevent crashes from callbacks after deallocation.

### **Dart PlatformView Widget**

```dart
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

/// Widget that embeds the native label view
class NativeLabel extends StatelessWidget {
  final String text;
  final Color textColor;
  final Map<String, dynamic> creationParams;
  
  NativeLabel({
    Key? key,
    required this.text,
    this.textColor = Colors.black,
  }) : creationParams = {
         'text': text,
         'textColor': '#${textColor.value.toRadixString(16).substring(2)}',
       },
       super(key: key);
  
  @override
  Widget build(BuildContext context) {
    if (Theme.of(context).platform == TargetPlatform.android) {
      // Android PlatformView using Texture Layer
      return AndroidView(
        viewType: 'com.example.app/native_label',
        // viewType must match the registered factory name on Android
        creationParams: creationParams,
        // Pass initial parameters to the platform view
        creationParamsCodec: const StandardMessageCodec(),
        // Codec used to encode creationParams
        onPlatformViewCreated: (id) {
          // Callback when view is created
          // id is the unique view identifier
          print('Android PlatformView created with id: $id');
          
          // Can create MethodChannel here to communicate with specific view
          final channel = MethodChannel(
            'com.example.app/native_view/$id',
          );
          channel.invokeMethod('setText', {'text': 'Updated from Dart'});
        },
      );
    } else if (Theme.of(context).platform == TargetPlatform.iOS) {
      // iOS PlatformView using UiKitView
      return UiKitView(
        viewType: 'com.example.app/native_label',
        creationParams: creationParams,
        creationParamsCodec: const StandardMessageCodec(),
        onPlatformViewCreated: (id) {
          print('iOS PlatformView created with id: $id');
        },
      );
    }
    
    return Text('PlatformView not supported on this platform');
  }
}
```

**Explanation:**

- **PlatformView widgets**: `AndroidView` and `UiKitView` are Flutter widgets that embed native platform views.
- **viewType**: String identifier that must match the registered factory name on the platform side.
- **creationParams**: Initial data passed to the platform view constructor. Must be encodable with the specified codec.
- **onPlatformViewCreated**: Callback fired when the native view is created. Receives the view ID which can be used to create MethodChannels for that specific view instance.
- **Platform checks**: Use `Theme.of(context).platform` to render different platform views for Android and iOS.

---

## **23.5 Texture and External Textures**

Texture widgets allow you to display images from native code (e.g., camera preview, video frames, OpenGL rendering) in Flutter using a texture ID.

### **Dart Implementation**

```dart
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

class CameraTextureWidget extends StatefulWidget {
  @override
  _CameraTextureWidgetState createState() => _CameraTextureWidgetState();
}

class _CameraTextureWidgetState extends State<CameraTextureWidget> {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/camera');
  
  Texture? _texture;
  int? _textureId;
  bool _isInitialized = false;
  
  @override
  void initState() {
    super.initState();
    _initializeCamera();
  }
  
  Future<void> _initializeCamera() async {
    try {
      // Request texture creation from platform
      // Platform returns a texture ID (int) that corresponds to a native texture
      final Map<dynamic, dynamic> result = await _channel.invokeMethod('create');
      // Result contains 'textureId' which is the handle to the native texture
      
      final int textureId = result['textureId'];
      final int width = result['width'];
      final int height = result['height'];
      
      setState(() {
        _textureId = textureId;
        // Create Flutter Texture widget using the texture ID
        _texture = Texture(textureId: textureId);
        // Texture widget displays the native texture associated with this ID
        _isInitialized = true;
      });
      
      // Start camera stream
      await _channel.invokeMethod('start', {'textureId': textureId});
      
    } on PlatformException catch (e) {
      print('Failed to initialize camera: ${e.message}');
      setState(() => _isInitialized = false);
    }
  }
  
  Future<void> _disposeCamera() async {
    if (_textureId != null) {
      await _channel.invokeMethod('dispose', {'textureId': _textureId});
      // Platform releases the native texture resources
    }
  }
  
  @override
  void dispose() {
    _disposeCamera();
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    if (!_isInitialized || _texture == null) {
      return Center(child: CircularProgressIndicator());
    }
    
    return AspectRatio(
      aspectRatio: 16 / 9,
      child: _texture!,  // Display the native camera texture
    );
  }
}
```

**Explanation:**

- **Texture widget**: Flutter widget that displays a native texture identified by an integer `textureId`.
- **Texture ID**: An integer handle that references a native graphics texture (OpenGL, Metal, or Vulkan texture) created by the platform.
- **Lifecycle**: 
  1. Dart requests texture creation → Platform creates texture and returns ID
  2. Dart creates Texture widget with that ID
  3. Platform renders to the texture (camera frames, video, OpenGL)
  4. Flutter composites the texture into the widget tree
  5. On dispose, Dart notifies platform to release texture resources
- **Use cases**: Camera preview, video playback, OpenGL rendering, maps, web content rendering.

### **Android Texture Implementation (Kotlin)**

```kotlin
package com.example.app

import android.content.Context
import android.graphics.SurfaceTexture
import android.hardware.camera2.*
import android.os.Handler
import android.os.HandlerThread
import android.view.Surface
import android.view.TextureView
import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.MethodChannel
import io.flutter.view.TextureRegistry

class MainActivity : FlutterActivity() {
    private val CHANNEL = "com.example.app/camera"
    private var cameraDevice: CameraDevice? = null
    private var captureSession: CameraCaptureSession? = null
    private var backgroundHandler: Handler? = null
    private var backgroundThread: HandlerThread? = null
    
    // Map to store texture entries by ID
    private val textureEntries = mutableMapOf<Long, TextureRegistry.SurfaceTextureEntry>()
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        MethodChannel(flutterEngine.dartExecutor.binaryMessenger, CHANNEL)
            .setMethodCallHandler { call, result ->
                when (call.method) {
                    "create" -> {
                        // Create a new texture entry
                        val entry = flutterEngine.renderer.createSurfaceTexture()
                        // createSurfaceTexture() creates a new SurfaceTextureEntry
                        // This allocates a new OpenGL texture ID that Flutter can render
                        
                        val textureId = entry.id()
                        // Get the unique texture ID (Long)
                        
                        // Store the entry so we can access the SurfaceTexture later
                        textureEntries[textureId] = entry
                        
                        // Return texture details to Dart
                        result.success(mapOf(
                            "textureId" to textureId,
                            "width" to 1920,  // Default width
                            "height" to 1080  // Default height
                        ))
                    }
                    
                    "start" -> {
                        val textureId = call.argument<Long>("textureId")
                        // Get texture ID from arguments
                        
                        if (textureId != null && textureEntries.containsKey(textureId)) {
                            val entry = textureEntries[textureId]
                            // Get the SurfaceTextureEntry
                            
                            val surfaceTexture = entry?.surfaceTexture()
                            // Get the SurfaceTexture object
                            
                            startCamera(textureId, surfaceTexture)
                            // Start camera preview rendering to this texture
                            
                            result.success(null)
                        } else {
                            result.error(
                                "INVALID_TEXTURE",
                                "Texture not found",
                                null
                            )
                        }
                    }
                    
                    "dispose" -> {
                        val textureId = call.argument<Long>("textureId")
                        
                        if (textureId != null) {
                            stopCamera()
                            // Stop camera before releasing texture
                            
                            textureEntries[textureId]?.release()
                            // Release the SurfaceTextureEntry
                            // This frees the OpenGL texture resources
                            
                            textureEntries.remove(textureId)
                            result.success(null)
                        } else {
                            result.error("INVALID_ARGUMENT", "Texture ID required", null)
                        }
                    }
                    
                    else -> result.notImplemented()
                }
            }
    }
    
    private fun startCamera(textureId: Long, surfaceTexture: SurfaceTexture?) {
        if (surfaceTexture == null) return
        
        // Setup background thread for camera operations
        backgroundThread = HandlerThread("CameraBackground").apply { start() }
        backgroundHandler = Handler(backgroundThread!!.looper)
        
        val cameraManager = getSystemService(Context.CAMERA_SERVICE) as CameraManager
        
        try {
            // Get back camera ID
            val cameraId = cameraManager.cameraIdList[0]
            
            // Create Surface from SurfaceTexture
            val surface = Surface(surfaceTexture)
            // Surface is what the camera renders to
            
            cameraManager.openCamera(cameraId, object : CameraDevice.StateCallback() {
                override fun onOpened(camera: CameraDevice) {
                    cameraDevice = camera
                    
                    // Create capture request
                    val requestBuilder = camera.createCaptureRequest(
                        CameraDevice.TEMPLATE_PREVIEW
                    )
                    requestBuilder.addTarget(surface)
                    // Add the Surface as target for preview frames
                    
                    // Create capture session
                    camera.createCaptureSession(
                        listOf(surface),
                        object : CameraCaptureSession.StateCallback() {
                            override fun onConfigured(session: CameraCaptureSession) {
                                captureSession = session
                                // Start repeating request for preview
                                session.setRepeatingRequest(
                                    requestBuilder.build(),
                                    null,
                                    backgroundHandler
                                )
                            }
                            
                            override fun onConfigureFailed(session: CameraCaptureSession) {
                                println("Camera configuration failed")
                            }
                        },
                        backgroundHandler
                    )
                }
                
                override fun onDisconnected(camera: CameraDevice) {
                    camera.close()
                    cameraDevice = null
                }
                
                override fun onError(camera: CameraDevice, error: Int) {
                    camera.close()
                    cameraDevice = null
                }
            }, backgroundHandler)
            
        } catch (e: Exception) {
            e.printStackTrace()
        }
    }
    
    private fun stopCamera() {
        captureSession?.close()
        cameraDevice?.close()
        backgroundThread?.quitSafely()
        
        captureSession = null
        cameraDevice = null
        backgroundThread = null
        backgroundHandler = null
    }
}
```

**Explanation:**

- **TextureRegistry.SurfaceTextureEntry**: Represents a texture entry in Flutter's renderer. `createSurfaceTexture()` allocates a new OpenGL texture and returns an entry.
- **SurfaceTexture**: Android class that captures frames from a data source (like camera) and makes them available as an OpenGL texture.
- **Surface**: Wrapper around SurfaceTexture that can be used as a target for camera preview.
- **Camera2 API**: Modern Android camera API that renders to Surfaces.
- **Lifecycle management**: Store texture entries in a map to access them later for starting/disposing. Always release textures when done to prevent memory leaks.
- **Background thread**: Camera operations should happen on a background thread (HandlerThread) to avoid blocking the UI.

### **iOS Implementation (Swift)**

```swift
import UIKit
import Flutter
import AVFoundation  // For camera

// UIView subclass to be embedded in Flutter
class NativeCameraView: UIView {
    private var captureSession: AVCaptureSession?
    private var previewLayer: AVCaptureVideoPreviewLayer?
    private var textureRegistry: FlutterTextureRegistry?
    private var textureId: Int64?
    
    init(frame: CGRect, textureRegistry: FlutterTextureRegistry) {
        self.textureRegistry = textureRegistry
        super.init(frame: frame)
        setupCamera()
    }
    
    required init?(coder: NSCoder) {
        super.init(coder: coder)
    }
    
    private func setupCamera() {
        captureSession = AVCaptureSession()
        captureSession?.sessionPreset = .high
        
        guard let backCamera = AVCaptureDevice.default(.builtInWideAngleCamera, for: .video, position: .back) else {
            print("Back camera not available")
            return
        }
        
        do {
            let input = try AVCaptureDeviceInput(device: backCamera)
            captureSession?.addInput(input)
            
            // Create texture for Flutter
            if let registry = textureRegistry {
                // Create a texture entry
                let textureEntry = registry.registerTexture()
                // registerTexture() returns a FlutterTextureRegistryEntry
                
                textureId = textureEntry.textureID()
                // Get the texture ID to pass to Dart
                
                // Setup output to texture
                let videoOutput = AVCaptureVideoDataOutput()
                videoOutput.setSampleBufferDelegate(
                    self,
                    queue: DispatchQueue(label: "videoQueue")
                )
                captureSession?.addOutput(videoOutput)
                
                // Start session
                captureSession?.startRunning()
                
                // Notify Flutter that texture is available
                textureEntry.textureFrameAvailable()
                // This notifies Flutter to render the texture
            }
            
        } catch {
            print("Error setting up camera: \(error)")
        }
    }
    
    deinit {
        captureSession?.stopRunning()
        if let id = textureId {
            textureRegistry?.unregisterTexture(id)
        }
    }
}

// Extension to handle video frames
extension NativeCameraView: AVCaptureVideoDataOutputSampleBufferDelegate {
    func captureOutput(
        _ output: AVCaptureOutput,
        didOutput sampleBuffer: CMSampleBuffer,
        from connection: AVCaptureConnection
    ) {
        // Process frame and update texture
        // This is where you would copy the pixel buffer to the Flutter texture
        if let textureId = textureId {
            textureRegistry?.textureFrameAvailable(textureId)
        }
    }
}

// Factory to create NativeCameraView instances
class NativeCameraViewFactory: NSObject, FlutterPlatformViewFactory {
    private var messenger: FlutterBinaryMessenger
    private var textureRegistry: FlutterTextureRegistry
    
    init(messenger: FlutterBinaryMessenger, textureRegistry: FlutterTextureRegistry) {
        self.messenger = messenger
        self.textureRegistry = textureRegistry
        super.init()
    }
    
    func create(
        withFrame frame: CGRect,
        viewIdentifier viewId: Int64,
        arguments args: Any?
    ) -> FlutterPlatformView {
        return NativeCameraView(
            frame: frame,
            textureRegistry: textureRegistry
        )
    }
    
    // Return the message codec for creation parameters
    func createArgsCodec() -> FlutterMessageCodec & NSObjectProtocol {
        return FlutterStandardMessageCodec.sharedInstance()
    }
}
```

**Explanation:**

- **FlutterTextureRegistry**: Registry for external textures. `registerTexture()` creates a new texture entry and returns a unique texture ID.
- **Texture ID**: An integer handle that Flutter uses to reference the native texture. Pass this to Dart to display the texture.
- **AVCaptureSession**: iOS camera capture session that outputs to the Flutter texture.
- **textureFrameAvailable()**: Notifies Flutter that a new frame is available in the texture and should be rendered.
- **Factory registration**: The factory must be registered in `AppDelegate` (not shown here but required) using `registrar.register(factory, withId: "viewType")`.

---

## **23.5 Best Practices and Architecture**

### **Error Handling and Timeouts**

```dart
import 'package:flutter/services.dart';

class PlatformChannelManager {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/service');
  
  /// Invokes platform method with timeout and retry logic
  static Future<T> invokeWithTimeout<T>(
    String method, {
    dynamic arguments,
    Duration timeout = const Duration(seconds: 5),
    int maxRetries = 3,
  }) async {
    int attempts = 0;
    
    while (attempts < maxRetries) {
      try {
        // Create a future with timeout
        final result = await _channel
            .invokeMethod<T>(method, arguments)
            .timeout(timeout);
        // .timeout() throws TimeoutException if operation takes too long
        
        return result as T;
      } on TimeoutException catch (e) {
        attempts++;
        if (attempts >= maxRetries) {
          throw PlatformException(
            code: 'TIMEOUT',
            message: 'Operation timed out after $maxRetries attempts',
            details: e.toString(),
          );
        }
        // Exponential backoff
        await Future.delayed(Duration(milliseconds: 100 * attempts));
      } on PlatformException catch (e) {
        // Don't retry on platform errors, just rethrow
        rethrow;
      }
    }
    
    throw PlatformException(code: 'UNKNOWN', message: 'Unexpected error');
  }
  
  /// Ensures platform resources are cleaned up even if errors occur
  static Future<void> safeDispose() async {
    try {
      await _channel.invokeMethod('dispose');
    } catch (e) {
      // Log but don't throw during cleanup
      print('Error during dispose: $e');
    }
  }
}
```

**Explanation:**

- **Timeout handling**: Use `.timeout()` on Futures to prevent indefinite waiting for platform responses.
- **Retry logic**: Implement exponential backoff for transient failures (timeouts) but not for logical errors.
- **Resource cleanup**: Always wrap dispose calls in try-catch to prevent cleanup failures from crashing the app.
- **Type safety**: Use generic type parameters `<T>` with `invokeMethod<T>` for type-safe returns.

### **Architecture Pattern: Platform Channel Abstraction**

```dart
// Abstract interface for platform-specific implementations
abstract class BiometricAuth {
  Future<bool> isAvailable();
  Future<bool> authenticate(String reason);
  Future<void> stopAuthentication();
}

// Factory to get appropriate implementation
class BiometricAuthFactory {
  static BiometricAuth get instance {
    if (Platform.isAndroid) {
      return _AndroidBiometricAuth();
    } else if (Platform.isIOS) {
      return _IOSBiometricAuth();
    }
    throw UnsupportedError('Platform not supported');
  }
}

// Android-specific implementation
class _AndroidBiometricAuth implements BiometricAuth {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/biometric');
  
  @override
  Future<bool> isAvailable() async {
    try {
      return await _channel.invokeMethod<bool>('isAvailable') ?? false;
    } on PlatformException catch (e) {
      print('Biometric check failed: ${e.message}');
      return false;
    }
  }
  
  @override
  Future<bool> authenticate(String reason) async {
    try {
      final result = await _channel.invokeMethod<bool>('authenticate', {
        'reason': reason,
        'options': {
          'stickyAuth': true,
          'sensitiveTransaction': true,
        },
      });
      return result ?? false;
    } on PlatformException catch (e) {
      if (e.code == 'UserCancel') {
        print('User cancelled authentication');
      } else if (e.code == 'NotAvailable') {
        print('Biometric not available');
      }
      return false;
    }
  }
  
  @override
  Future<void> stopAuthentication() async {
    await _channel.invokeMethod('stopAuthentication');
  }
}

// iOS-specific implementation
class _IOSBiometricAuth implements BiometricAuth {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/biometric');
  
  @override
  Future<bool> isAvailable() async {
    // iOS-specific implementation details...
    return await _channel.invokeMethod<bool>('isAvailable') ?? false;
  }
  
  @override
  Future<bool> authenticate(String reason) async {
    // iOS-specific implementation with LocalAuthentication framework...
    return await _channel.invokeMethod<bool>('authenticate', {
      'reason': reason,
    }) ?? false;
  }
  
  @override
  Future<void> stopAuthentication() async {
    await _channel.invokeMethod('stopAuthentication');
  }
}

// Usage in Flutter
class AuthScreen extends StatelessWidget {
  final BiometricAuth _auth = BiometricAuthFactory.instance;
  
  Future<void> _authenticate() async {
    final available = await _auth.isAvailable();
    if (!available) {
      print('Biometric authentication not available');
      return;
    }
    
    final success = await _auth.authenticate('Please authenticate to access');
    if (success) {
      print('Authentication successful');
      // Navigate to secure area
    } else {
      print('Authentication failed');
    }
  }
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Biometric Auth')),
      body: Center(
        child: ElevatedButton(
          onPressed: _authenticate,
          child: Text('Authenticate'),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Abstraction layer**: Create an abstract interface (`BiometricAuth`) that defines the contract, hiding platform-specific details from the UI layer.
- **Factory pattern**: Use a factory to instantiate the correct implementation based on `Platform.isAndroid` or `Platform.isIOS`.
- **Platform-specific implementations**: Separate classes for Android and iOS that handle the MethodChannel calls with platform-specific logic.
- **Error abstraction**: Convert platform-specific error codes into domain-specific exceptions or boolean results.
- **Testability**: The abstraction allows mocking the biometric service for unit tests without relying on the platform.
- **UI decoupling**: The UI only knows about the abstract `BiometricAuth` interface, not the MethodChannel implementation details.

---

## **Chapter Summary**

In this chapter, we covered Platform Channels for integrating native platform functionality with Flutter:

### **Key Takeaways:**

1. **Platform Channels**: Bridge between Dart and native code (Android/iOS). Three types:
   - **MethodChannel**: Request-response pattern for method invocation
   - **EventChannel**: Streaming data from platform to Dart
   - **BasicMessageChannel**: Custom message passing with specific codecs

2. **MethodChannel**:
   - Create with unique channel name (reverse domain notation)
   - Use `invokeMethod()` to call native methods
   - Handle `PlatformException` for errors
   - Implement on platform side using `setMethodCallHandler`

3. **EventChannel**:
   - Use for continuous data streams (sensors, location, native events)
   - Implement `StreamHandler` on platform side
   - Use `EventSink` to send events to Dart
   - Always clean up resources in `onCancel`

4. **PlatformView**:
   - Embed native Android Views or iOS UIViews in Flutter widget tree
   - Implement `PlatformView` (Android) or `FlutterPlatformView` (iOS)
   - Use `AndroidView` or `UiKitView` widgets in Flutter
   - Create per-view MethodChannels for controlling individual native views

5. **Texture**:
   - Display native OpenGL textures in Flutter using `Texture` widget
   - Platform creates texture and returns ID to Dart
   - Platform renders camera/video/OpenGL content to texture
   - Flutter composites texture as part of widget tree
   - More performant than PlatformView for video/camera content

6. **Best Practices**:
   - Always use unique channel names (reverse domain notation)
   - Handle timeouts and retries for platform calls
   - Clean up resources (cancel streams, release textures) in dispose
   - Abstract platform channels behind service interfaces for testability
   - Use appropriate thread handling (background threads for heavy work)
   - Validate arguments and return proper error codes

### **Next Steps:**

The next chapter will cover **Chapter 24: Native Android Integration**, including:
- Android Activity and Fragment embedding
- Kotlin/Java interop with Flutter
- Android-specific plugins and manifest configuration
- App links and intent handling
- Background services and WorkManager

---

**End of Chapter 23**

---

# **Next Chapter: Chapter 24 - Native Android Integration**

Chapter 24 will explore deep integration with the Android platform, covering Activity lifecycle management, Fragment embedding, background processing, and Android-specific APIs like Intents, Services, and WorkManager.